In [1]:
.L ../Build/Combustion.so

In [2]:
#include "../Simulation.hpp"
#include "../Solvers/Gas/GasSolvers.hpp"
#include "../Solvers/Gas/ExactSteadySolver.hpp"
#include "../Solvers/Rocket/RocketSolver.hpp"
#include "../Chemistry/Reaction.hpp"
#include "../Math/Vector.hpp"
#include "../CPGF/Plot2d/LinePlot.hpp"
#include <iostream>
#include <iomanip>
#include <thread>
#include <cmath>

In [3]:
using namespace Chemistry;

In [4]:
using namespace Math;

In [5]:
using namespace Utilities;

In [6]:
using namespace CPGF::Plot2d;

In [7]:
using namespace CPGF;

In [5]:
{
    SolidGasReaction QR(0, 0, 718, 287, 0, 0, 101325, 0, SolidGasReactionModel::F0);
    std::cout << QR.to_string() << "\n";
}

Begin SolidGasReaction
	k_s = 0.000000
	cV_s = 0.000000
	cV_g = 718.000000
	R_g = 287.000000
	γ = 1.399721
	E_a = 0.000000
	P_0 = 101325.000000
	ΔH = 0.000000
	model = F0
End SolidGasReaction


In [6]:
{
    SolidGasReaction QR(0, 0, 718, 287, 0, 0, 101325, 0, SolidGasReactionModel::F0);
    Cell C(CellType::GAS, Vector<double>({0,0,0}), 0, 1, 1, QR);
    std::cout << C.to_string() << "\n";
}

Begin Cell
	address = 0x7fc8e10efbc0
	left_neighbour = (nil)
	right_neighbour = (nil)
	cell_type = GAS
	U = (0.000000, 0.000000, 0.000000)
	a = 0.000000
	b = 1.000000
	A = 1.000000
Begin SolidGasReaction
	k_s = 0.000000
	cV_s = 0.000000
	cV_g = 718.000000
	R_g = 287.000000
	γ = 1.399721
	E_a = 0.000000
	P_0 = 101325.000000
	ΔH = 0.000000
	model = F0
End SolidGasReaction
	ρ = 0.000000
	c = -nan
	ρ_s = 0.000000
	w_s = -nan
	ρ_g = 0.000000
	w_g = -nan
	v = 0.000000
	Q_dot = 0.000000
	T = -nan
	P = -nan
	H = -nan
	E = -nan
End Cell


In [9]:
{
    Mesh mesh;
    std::cout << mesh.detail_subdivide_threshold << "\n";
    SolidGasReaction QR(0, 0, 718, 287, 0, 0, 101325, 0, SolidGasReactionModel::F0);
    Cell* L = nullptr;
    Cell* C = nullptr;
    Cell* first = nullptr;
    for (unsigned int i = 0; i < 10; i++)
    {
        L = C;
        C = new Cell(CellType::GAS, Vector<double>({0,0,0}), 0, 1, 1, QR);
        if (i > 0)
        {
            C->left_neighbour = L;
            L->right_neighbour = C;
        }
        else
        {
            first = C;
        }
    }
    mesh = Mesh(first, C);
}

0.1


In [18]:
{
    std::cout << (unsigned int)((CellType)0);
}

0

Tests with mesh arrays

In [ ]:
{
    FileArray<Mesh> mesh("hola3.dat");
    Mesh instant_mesh;
    mesh.push_back(instant_mesh);
}

Values calculator

In [27]:
{
    double M = 0.2999944686889648;
    double A = 4.0702;
    double rho_0 = 2.32;
    double P_0 = 200000;
    double T_0 = 300;
    double gamma = 1.4;
    
    double T = T_0 / (1 + (gamma-1)/2*M*M);
    double P = P_0 / pow(1 + (gamma-1)/2*M*M, gamma/(gamma - 1));
    double rho = rho_0 / pow(1 + (gamma-1)/2*M*M, 1/(gamma - 1));
    double v = M * sqrt(gamma*P/rho);
    double F = rho*v*A;
    double H = gamma / (gamma - 1) * P_0/rho_0;
    
    std::cout << std::setprecision(16) << "T = " << T << ", P = " << P << ", rho = " << rho << ", v = " << v << ", F = " << F
        << ", H = " << H << "\n";
}

T = 294.6956734819336, P = 187894.3684832251, rho = 2.218805571849392, v = 103.2938638373808, F = 932.845070324472, H = 301724.1379310346


In [26]:
{
    double M = 1;
    double A = 2;
    double rho_0 = 2.32;
    double P_0 = 200000;
    double T_0 = 300;
    double gamma = 1.4;
    
    Solvers::Gas::ExactSteadySolver SS(rho_0, P_0, T_0, M, A, gamma, Solvers::Gas::SolutionType::SUBSONIC);
    
    std::cout << SS.M(4.0702) << "\n";
}

0.2999944686889648


In [11]:
{
    double A_c = M_PI*0.025*0.025;
    double A_e = A_c;
    Solvers::Rocket::SteadySolver SS(1423.681257, 0.071912799, 0.325, 1.218878249, 160, A_c);
    SS.solve_for_exit_area(A_e);
    std::cout << SS.to_string() << "\n";
}

Begin Steady Rocket Solver.
	— Input:
		• chamber Mach number at atmospheric pressure = 0.071913
		• adiabatic gas constant = 1.218878
		• gas constant = 160.000000 J/(kg·K)
		• power law exponent n = 0.325000
		• chamber temperature = 1423.681257 K
	— Chamber data:
		• chamber area = 0.001963 m²
		• chamber pressure = 101325.008203 Pa
		• chamber speed = 37.892452 m/s
		• chamber Mach number = 0.071913
		• chamber gas density = 0.444820 kg/m³
	— Exit data
		• exit area = 0.001963 m²
		• exit pressure = 101325.000000 Pa
		• exit temperature = 1423.681236 K
		• exit speed = 37.892939 m/s
		• exit Mach number = 0.071914
		• exit density = 0.444820 kg/m³
		• mass flow = 0.033096 kg/s
		• thrust = 1.254095 N
End Steady Rocket Solver.


In [6]:
{
    double A_c = M_PI*0.025*0.025;
    double A_e = A_c/2;
    Solvers::Rocket::SteadySolver SS(1423.681257, 0.071912799, 0.325, 1.218878249, 160, A_c);
    SS.solve_for_exit_area(A_e);
    std::cout << SS.to_string() << "\n";
}

Begin Steady Rocket Solver.
	— Input:
		• chamber Mach number at atmospheric pressure = 0.071913
		• adiabatic gas constant = 1.218878
		• gas constant = 160.000000 J/(kg·K)
		• power law exponent n = 0.325000
		• chamber temperature = 1423.681257 K
	— Chamber data:
		• chamber area = 0.001963 m²
		• chamber pressure = 102296.122569 Pa
		• chamber speed = 37.649265 m/s
		• chamber Mach number = 0.071451
		• chamber gas density = 0.449083 kg/m³
	— Exit data
		• exit area = 0.000982 m²
		• exit pressure = 101325.000000 Pa
		• exit temperature = 1421.244750 K
		• exit speed = 75.890201 m/s
		• exit Mach number = 0.144149
		• exit density = 0.445582 kg/m³
		• mass flow = 0.033198 kg/s
		• thrust = 2.519411 N
End Steady Rocket Solver.


In [7]:
{
    double A_c = M_PI*0.025*0.025;
    double A_e = A_c/2;
    Solvers::Rocket::SteadySolver SS(1423.681257, 0.071912799, 0.325, 1.218878249, 160, A_c);
    SS.calculate_optimum_parameters();
    std::cout << SS.to_string() << "\n";
}

Begin Steady Rocket Solver.
	— Input:
		• chamber Mach number at atmospheric pressure = 0.071913
		• adiabatic gas constant = 1.218878
		• gas constant = 160.000000 J/(kg·K)
		• power law exponent n = 0.325000
		• chamber temperature = 1423.681257 K
	— Chamber data:
		• chamber area = 0.001963 m²
		• chamber pressure = 180409.119315 Pa
		• chamber speed = 25.670656 m/s
		• chamber Mach number = 0.048718
		• chamber gas density = 0.792001 kg/m³
	— Exit data
		• exit area = 0.000162 m²
		• exit pressure = 101325.000000 Pa
		• exit temperature = 1283.577463 K
		• exit speed = 500.323839 m/s
		• exit Mach number = 1.000000
		• exit density = 0.493372 kg/m³
		• mass flow = 0.039920 kg/s
		• thrust = 19.973022 N
End Steady Rocket Solver.


In [12]:
{
    double A_c = M_PI*0.05*0.05;
    double A_e = A_c;
    Solvers::Rocket::SteadySolver SS(1423.681257, 0.071912799, 0.325, 1.218878249, 160, A_c);
    SS.solve_for_exit_area(A_e);
    std::cout << SS.to_string() << "\n";
}

Begin Steady Rocket Solver.
	— Input:
		• chamber Mach number at atmospheric pressure = 0.071913
		• adiabatic gas constant = 1.218878
		• gas constant = 160.000000 J/(kg·K)
		• power law exponent n = 0.325000
		• chamber temperature = 1423.681257 K
	— Chamber data:
		• chamber area = 0.007854 m²
		• chamber pressure = 101325.008203 Pa
		• chamber speed = 37.892452 m/s
		• chamber Mach number = 0.071913
		• chamber gas density = 0.444820 kg/m³
	— Exit data
		• exit area = 0.007854 m²
		• exit pressure = 101325.000000 Pa
		• exit temperature = 1423.681236 K
		• exit speed = 37.892939 m/s
		• exit Mach number = 0.071914
		• exit density = 0.444820 kg/m³
		• mass flow = 0.132383 kg/s
		• thrust = 5.016379 N
End Steady Rocket Solver.


In [8]:
{
    double A_c = M_PI*0.05*0.05;
    double A_e = A_c/10;
    Solvers::Rocket::SteadySolver SS(1423.681257, 0.071912799, 0.325, 1.218878249, 160, A_c);
    SS.solve_for_exit_area(A_e);
    std::cout << SS.to_string() << "\n";
}

Begin Steady Rocket Solver.
	— Input:
		• chamber Mach number at atmospheric pressure = 0.071913
		• adiabatic gas constant = 1.218878
		• gas constant = 160.000000 J/(kg·K)
		• power law exponent n = 0.325000
		• chamber temperature = 1423.681257 K
	— Chamber data:
		• chamber area = 0.007854 m²
		• chamber pressure = 145129.156593 Pa
		• chamber speed = 29.732200 m/s
		• chamber Mach number = 0.056426
		• chamber gas density = 0.637121 kg/m³
	— Exit data
		• exit area = 0.000785 m²
		• exit pressure = 101325.000000 Pa
		• exit temperature = 1334.726848 K
		• exit speed = 399.250248 m/s
		• exit Mach number = 0.782544
		• exit density = 0.474465 kg/m³
		• mass flow = 0.148778 kg/s
		• thrust = 59.399734 N
End Steady Rocket Solver.


In [9]:
{
    double A_c = M_PI*0.05*0.05;
    double A_e = A_c*10;
    Solvers::Rocket::SteadySolver SS(1423.681257, 0.071912799, 0.325, 1.218878249, 160, A_c);
    SS.solve_for_exit_area(A_e);
    std::cout << SS.to_string() << "\n";
}

Begin Steady Rocket Solver.
	— Input:
		• chamber Mach number at atmospheric pressure = 0.071913
		• adiabatic gas constant = 1.218878
		• gas constant = 160.000000 J/(kg·K)
		• power law exponent n = 0.325000
		• chamber temperature = 1423.681257 K
	— Chamber data:
		• chamber area = 0.007854 m²
		• chamber pressure = 101008.078644 Pa
		• chamber speed = 37.972664 m/s
		• chamber Mach number = 0.072065
		• chamber gas density = 0.443428 kg/m³
	— Exit data
		• exit area = 0.078540 m²
		• exit pressure = 101325.000000 Pa
		• exit temperature = 1424.482367 K
		• exit speed = 3.787495 m/s
		• exit Mach number = 0.007186
		• exit density = 0.444569 kg/m³
		• mass flow = 0.132246 kg/s
		• thrust = 0.500880 N
End Steady Rocket Solver.


In [10]:
{
    double A_c = M_PI*0.05*0.05;
    double A_e = A_c/10;
    Solvers::Rocket::SteadySolver SS(1423.681257, 0.071912799, 0.325, 1.218878249, 160, A_c);
    SS.calculate_optimum_parameters();
    std::cout << SS.to_string() << "\n";
}

Begin Steady Rocket Solver.
	— Input:
		• chamber Mach number at atmospheric pressure = 0.071913
		• adiabatic gas constant = 1.218878
		• gas constant = 160.000000 J/(kg·K)
		• power law exponent n = 0.325000
		• chamber temperature = 1423.681257 K
	— Chamber data:
		• chamber area = 0.007854 m²
		• chamber pressure = 180409.119315 Pa
		• chamber speed = 25.670656 m/s
		• chamber Mach number = 0.048718
		• chamber gas density = 0.792001 kg/m³
	— Exit data
		• exit area = 0.000647 m²
		• exit pressure = 101325.000000 Pa
		• exit temperature = 1283.577463 K
		• exit speed = 500.323839 m/s
		• exit Mach number = 1.000000
		• exit density = 0.493372 kg/m³
		• mass flow = 0.159681 kg/s
		• thrust = 79.892090 N
End Steady Rocket Solver.
